# Estimation of Supply

We estimated the required number(=supply) of EV ports in Georgia based on the following assumptions and process.

### Assumptions
1. The total number of vehicles will increase at a rate similar to the trend over the past 10 years.
2. The proportion of EVs within the total vehicle count for each region type (Urban, Suburban, Rural) will maintain similar trends in the near future.

### Supply Estimation Process
1. Using the TIGER/Line Urban polygon data, we classified the counties in Georgia into Urban, Suburban, and Rural regions.
2. Extract the number of vehicles in each county of Georgia from the provided data (2019, HT index).
3. Predict the total number of vehicles in Georgia for the year 2025, considering the publicly available U.S. total vehicle growth rate and Georgia's average population growth rate compared to the national average.
4. Predict the total trips of vehicles in Georgia by reflecting traffic patterns from OD data, and then scaling it by dividing by the trip ratio per vehicle to align with the total number of vehicles.
5. Using the 'EERE Report: National Plug-In Electric Vehicle' as a reference, apply the EV ratio for each region (Urban, Suburban, Rural) to predict the number of EVs in each county of Georgia.
6. Apply the optimized EV port criteria for each region, referenced from the 'EERE Report: National Plug-In Electric Vehicle', to estimate the number of EV charging ports(Lv2. fast & DCFC) required for each county in Georgia.
   - Processed Data: 'county_trip_ev_ports_lv2.shp', 'county_trip_ev_ports_DCFC.shp'
7. Distribute the estimated number of EV ports within each county to the Urban and Suburban polygons, while assigning the remaining EV ports to the Rural counties as is.
   - Data: 'lv2_supply_polygon_urban.shp', 'DCFC_supply_polygon_urban.shp'


In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.ops import unary_union

## Predicting Total Vehicle Count in Georgia by 2025

In [ ]:
# TODO: Path to the Census Block Group polygon shapefile (e.g., "/path/to/census_block_group.shp")

# Step 1: Estimating Total Vehicle Count in Georgia by 2025

# Load the vehicle registration data ('/path/to/htaindex2019_data_counties_13.csv'))
file_path = '/home/sehoon/Desktop/ACM-SIGSPATIAL-Cup-2024/for_git_240920/Data/htaindex2019_data_counties_13.csv'
df = pd.read_csv(file_path)

# Calculate 'registered_car' (= households * autos per household)
df['registed_car'] = df['households'] * df['autos_per_hh_ami']

# Estimate vehicle growth rate based on U.S. and Georgia population growth from census report 2021
us_vehicle_growth_rate = 0.015  # 1.5% average U.S. growth rate 
us_population_growth = 7.4 / 100  # U.S. population growth (2010-2020) - from census report
georgia_population_growth = 10.6 / 100  # Georgia population growth (2010-2020) - from census report
georgia_vehicle_growth_rate = us_vehicle_growth_rate * (georgia_population_growth / us_population_growth)

# Predict vehicle count from 2020 to 2025
for year in range(2020, 2026):
    df[f'{year}_car'] = df['registed_car'] * (1 + georgia_vehicle_growth_rate) ** (year - 2019)


# Step 2: Classifying Counties into Urban, Suburban, and Rural Categories

# Load the Georgia Urban Area shapefile (e.g., '/path/to/tl_rd22_us_uac20.shp')
tl_rd22_file_path = None
tl_rd22_gdf = gpd.read_file(tl_rd22_file_path)

# Remove suburban polygons that extend beyond the Georgia state boundary
georgia_urban_area_gdf = tl_rd22_gdf[
    tl_rd22_gdf['NAMELSAD20'].str.contains("GA Urban Area") & 
    ~tl_rd22_gdf['NAMELSAD20'].str.contains("Eufaula, AL--GA Urban Area") | 
    tl_rd22_gdf['NAMELSAD20'].isin(['Columbus, GA--AL Urban Area', 'Augusta-Richmond County, GA--SC Urban Area'])
]

# Load the county shapefile (e.g., '/path/to/tl_2023_us_county.shp')
county_shp_file_path = None
county_gdf = gpd.read_file(county_shp_file_path)

# Identify counties that intersect with Atlanta Urban Area or other Urban Areas
atlanta_urban_area = georgia_urban_area_gdf[georgia_urban_area_gdf['NAMELSAD20'] == 'Atlanta, GA Urban Area']

# Assign region type (Urban, Suburban, Rural) based on spatial intersection
def assign_region(row, atlanta_urban_area, georgia_urban_area_gdf):
    if row.geometry.intersects(atlanta_urban_area.geometry.unary_union):
        return 'Urban'
    elif row.geometry.intersects(georgia_urban_area_gdf.geometry.unary_union):
        return 'Suburban'
    else:
        return 'Rural'

county_gdf['Region'] = county_gdf.apply(assign_region, atlanta_urban_area=atlanta_urban_area, georgia_urban_area_gdf=georgia_urban_area_gdf, axis=1)

# Step 3: Predicting 2025 Vehicle Count for Each County

# Mapping the estimated number of vehicles for 2025 to each county through georeferencing.
county_gdf['GEOID'] = county_gdf['GEOID'].astype(str)
county_gdf['2025_car'] = county_gdf['GEOID'].map(df_csv['2025_car'])


## Estimating EVs Count in Georgia by 2025


In [10]:
# ----------------Estimating EVs Count in Georgia by 2025 -------------------------------

# Step 4: Estimating the Number of EVs and Assigning EV Ports with OD Trip Data
# - Reference: EERE Report-National Plug-In Electric Vehicle

# Load OD trip data from given NHTS NextGen Georgia OD Data (e.g., '/path/to/OD_county.csv')
trip_csv_file = None
df_trip = pd.read_csv(trip_csv_file)
county_gdf = county_gdf.merge(df_trip[['county_NM', 'trip']], left_on='NAMELSAD', right_on='county_NM', how='left')

# Calculate total_trip and total_predicted_car
total_trip = county_gdf['trip'].sum()
total_predicted_car = county_gdf['2025_car'].sum()

# Calculate average_trip_per_car
average_trip_per_car = total_trip / total_predicted_car

# Add 'trip_based_car' column based on trip data
county_gdf['trip_based_car'] = county_gdf['trip'] / average_trip_per_car

# Weighted combination of 2025_car and trip_based_car (0.5:0.5)
weight_2025_car = 0.5
weight_trip_based = 0.5

def calculate_ev_car(row, w_a=weight_2025_car, w_b=weight_trip_based):
    # Weighted combination of 2025_car and trip_based_car
    weighted_value = (row['2025_car'] * w_a) + (row['trip_based_car'] * w_b)
    
    # - Reference: EERE Report-National Plug-In Electric Vehicle
    if row['Region'] == 'Rural':
        return round(weighted_value * 0.005)  # 0.5% for Rural
    elif row['Region'] == 'Urban':
        return round(weighted_value * 0.05)   # 5% for Urban
    elif row['Region'] == 'Suburban':
        return round(weighted_value * 0.015)  # 1.5% for Suburban
    else:
        return 0

county_gdf['EV_car'] = county_gdf.apply(calculate_ev_car, axis=1)

def calculate_ev_port(row):
    # - Reference: EERE Report-National Plug-In Electric Vehicle
    if row['Region'] == 'Urban':
        return round(row['EV_car'] * 36 / 1000)
    elif row['Region'] == 'Suburban':
        return round(row['EV_car'] * 54 / 1000)
    elif row['Region'] == 'Rural':
        return round(row['EV_car'] * 79 / 1000)
    else:
        return 0

county_gdf['EV_port'] = county_gdf.apply(calculate_ev_port, axis=1)

# Save the result with adjusted EV ports to one of the key final files (e.g., '/path/to/output/county_with_trip_ev_port_lv2.shp')
output_file_county_with_ev_ports = None
county_gdf.to_file(output_file_county_with_ev_ports)


# Step5. Print and check EV_port estimate result

# gdf.to_file(output_file)
gdf = gpd.read_file(output_file_county_with_ev_ports)

print("The process is complete. The 'EV_port' values have been updated according to the POI's 'supply' values.")

# Calculate statistics by region
for region in ['Urban', 'Suburban', 'Rural']:
    
    # Filter gdf by region
    region_gdf = gdf[gdf['Region'] == region]
    
    # Calculate EV_port statistics
    ev_port_summary = region_gdf['EV_port'].describe()
    total_ev_port = region_gdf['EV_port'].sum()

    # Output results
    print(f"\nEV_port statistics summary for {region} region:")
    print(ev_port_summary)
    print(f"\nTotal EV_ports for {region} region: {total_ev_port:.0f}")


The process is complete. The 'EV_port' values have been updated according to the POI's 'supply' values.

EV_port statistics summary for Urban region:
count      19.000000
mean      354.421053
std       364.748211
min        78.000000
25%       121.500000
50%       179.000000
75%       311.000000
max      1306.000000
Name: EV_port, dtype: float64

Total EV_ports for Urban region: 6734

EV_port statistics summary for Suburban region:
count     82.000000
mean      20.585366
std       20.153700
min        3.000000
25%        8.250000
50%       12.500000
75%       23.000000
max      113.000000
Name: EV_port, dtype: float64

Total EV_ports for Suburban region: 1688

EV_port statistics summary for Rural region:
count    58.000000
mean      3.224138
std       2.060856
min       0.000000
25%       2.000000
50%       3.000000
75%       4.000000
max      10.000000
Name: EV_port, dtype: float64

Total EV_ports for Rural region: 187


## Step 5. Split the provided urban shapefile into polygons (Using a GIS program)

- The process of distributing EV_ports, which were initially allocated by county, to Urban polygons, Suburban polygons, and Rural counties
- __The result will be the required supply (number of LV2 EV ports) for the Atlanta area ('lv2_supply_polygon_urban.shp') and other urban areas ('lv2_supply_polygon_suburban.shp')__

In [ ]:
# Step 6: Combining Urban and Suburban polygon 
georgia_urban_polygon = None # (e.g., '/path/to/georgia_urban_combined.shp' )
combined_gdf = gpd.read_file(georgia_urban_polygon)

# Step 7: Distribute EV Ports(county) by Area(urban & suburban polygon) and Save
for county_idx, county_row in county_gdf.iterrows():
    intersecting_combined = combined_gdf[combined_gdf.intersects(county_row.geometry)]
    if not intersecting_combined.empty:
        total_area_in_county = intersecting_combined.intersection(county_row.geometry).area.sum()
        for idx, combined_row in intersecting_combined.iterrows():
            overlap_area = combined_row.geometry.intersection(county_row.geometry).area
            area_ratio = overlap_area / total_area_in_county if total_area_in_county != 0 else 0
            combined_gdf.at[idx, 'EV_port'] += round(county_row['EV_port'] * area_ratio)

# Step 8: Separate Urban and Suburban into Individual Files
urban_gdf = combined_gdf[combined_gdf['NAMELSAD20'].str.contains("Atlanta", na=False)]
suburban_gdf = combined_gdf[~combined_gdf['NAMELSAD20'].str.contains("Atlanta", na=False)]

# Save Urban and Suburban shapefiles
urban_output_file = None # (e.g., '/path/to/lv2_supply_polygon_urban.shp' )
suburban_output_file = None # (e.g., '/path/to/lv2_supply_polygon_suburban.shp')

# Saving the urban and suburban shapefiles
urban_gdf.to_file(urban_output_file)
suburban_gdf.to_file(suburban_output_file)

print(f"Urban regions saved to {urban_output_file}")
print(f"Suburban regions saved to {suburban_output_file}")